In [9]:

import os
import pathlib
from datetime import datetime,timedelta

import numpy as np
import dash
from dash import dash_table, Input, Output, State, html, dcc, callback
import plotly.graph_objs as go
import  plotly.express as px
import dash_daq as daq
import dash_mantine_components as dmc
import pandas as pd
import pyodbc
import os
import matplotlib .pyplot as plt
from matplotlib.ticker import PercentFormatter
import base64
from PIL import Image
import plotly.figure_factory as ff
from itertools import cycle

def set_initial_datetime():
    current_datetime = datetime.now()
    
    # Reformat the datetime
    formatted_datetime = current_datetime.strftime("%Y-%m-%d %H:%M:%S")
    
    # Convert the reformatted datetime back to datetime format
    datetime_now = datetime.strptime(formatted_datetime, "%Y-%m-%d %H:%M:%S")
    datetime_now=pd.Timestamp(datetime_now)
    
    lastweek = datetime_now-timedelta(days=30)
    formatted_lastweek =  lastweek.strftime("%Y-%m-%d %H:%M:%S")
    lastweek = datetime.strptime(formatted_lastweek, "%Y-%m-%d %H:%M:%S")
    lastweek=pd.Timestamp(lastweek)
    return datetime_now,lastweek

datetime_now,lastweek = set_initial_datetime()

def sql_connection(datetime_start,datetime_end):
    
    
    
    driver_name = ''
    driver_names = [x for x in pyodbc.drivers() if x.endswith(' for SQL Server')]
    if driver_names:
        driver_name = driver_names[0]
        print(driver_name)
    if driver_name:
        conn_str = 'DRIVER={}; ...'.format(driver_name)
    # then continue with ...
    # pyodbc.connect(conn_str)
    # ... etc.
    else:
        print('(No suitable driver found. Cannot connect.)')


    #-------------------------------------- SQL server connection---------------------------
    # use ? as the parameter marker in order to pass dateTime variable to sql string
    from datetime import datetime, timedelta

    def count_hours(datetime_start, datetime_end):
        total_hours = 0
        current_date = datetime_start
        one_hour = timedelta(hours=1)
    
        while current_date <    datetime_end:
            if current_date.weekday() != 5:  # Exclude Saturdays
                if current_date.weekday() == 6 and current_date.time() >= datetime.strptime('19:00', '%H:%M').time():
                    pass  # Skip hours between Sunday 19:00 and Saturday 19:00
                else:
                    total_hours += 1  # Increment total hours
            current_date += one_hour
    
        return total_hours

        # Get the current datetime
  
    
    # date_from = '2024-03-21'
    # Time_from ='07:00:00'
    # dateTime_from = pd.to_datetime(date_from+" "+Time_from, infer_datetime_format=True)
    # 
    # date_to = '2024-03-22'
    # Time_to ='07:00:00'
    # dateTime_to = pd.to_datetime(date_to+" "+Time_to, infer_datetime_format=True)
    total_hours = count_hours(datetime_start, datetime_end)

    print("Total hours between the two dates (excluding Saturdays from 19:00 to Sundays 19:00):", total_hours)
    
    conn= pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER=GH-TQY-DB01;DATABASE=FAB_Production;UID=qry;PWD=wideband');
    #create tables-------------------------------------------------------
    #----------------------------------------------Scrap table-----------------------------------------
    scrapcodes = pd.read_sql_query('''SELECT * FROM scrapcodes''', con=conn)
    scrapcodes=scrapcodes.rename(columns={"desc4" : "scrap_description"})

    scrap = pd.read_sql_query('''SELECT * FROM scrap 
    WHERE rigdate BETWEEN ? AND ?''', con=conn,params=[datetime_start, datetime_end])
    scrap["DateTime_start"] = pd.to_datetime(scrap['rigdate'].apply(str)+" "+scrap['StartTime'], infer_datetime_format=True)
    scrap["DateTime_end"] = pd.to_datetime(scrap['rigdate'].apply(str)+" "+scrap['rigtime'], infer_datetime_format=True)
    scrap =scrap.query('(DateTime_start>=@datetime_start and DateTime_start<=@datetime_end)')

    scrap = pd.merge(scrap,scrapcodes,left_on= scrap["scrap1"].str.lower(),right_on=scrapcodes["scrapcode"].str.lower(), how='outer')
    # ----------------------------------------------Downtime table------------------------------------------------
    
    downtime = pd.read_sql_query('''SELECT * FROM downtime''', con=conn)
    downtime["DateTime_start"] = pd.to_datetime(downtime['rigdate'].apply(str)+" "+downtime['StartTime'], infer_datetime_format=True)
    downtime["DateTime_end"] = pd.to_datetime(downtime['rigdate'].apply(str)+" "+downtime['rigtime'], infer_datetime_format=True)
    downtime =downtime.query('(DateTime_start>=@datetime_start and DateTime_start<=@datetime_end)')
    
    # # ---------------------------------------------- Measurement table-------------------------------------------------
    # 
    # testInfo = pd.read_sql_query('''SELECT * FROM measinfo''', con=conn)
    # SpecMain = pd.read_sql_query('''SELECT * FROM vwLatestSpecs''', con=conn)
    # measurement_WDM= pd.read_sql_query('''SELECT * FROM measdata1x2''', con=conn)
    # measurement_TAP10= pd.read_sql_query('''SELECT * FROM measdata2x2''', con=conn)
    # measurement_3dB= pd.read_sql_query('''SELECT * FROM MeasRangeData''', con=conn)
    # 
    # MeasMain = pd.read_sql_query('''SELECT * FROM MeasMain''', con=conn)
    # parts_number = pd.read_sql_query('''SELECT * FROM parts''', con=conn)
    # 
    # 
    # measurement_data = pd.read_sql_query('''SELECT m.*,FuseOperator, FuseRig, FuseDate FROM (SELECT SerialNo, TestType,Devicecode, 'N x N' AS DeviceType, TestRig, TestDate, TestTime, Operator, ItemNo, ItemNoRev, Passed 
    # FROM measmain 
    # UNION SELECT serialno, testtype, devicecode, devicetype, rig, testdate, 
    # testtime, operator, spec AS itemno, itemnorev, passed 
    # FROM measinfo) m
    # LEFT JOIN  (SELECT serialno, operator AS FuseOperator, rig AS FuseRig, 
    # rigdate AS FuseDate
    # FROM rigdata) r ON m.serialno = r.serialno
    # WHERE TestDate BETWEEN ? AND ?
    # 
    # AND (devicecode LIKE 'T%' OR devicecode LIKE 'S%' OR devicecode LIKE 'F%' OR devicecode LIKE 'U%')  
    # ORDER BY testrig, testdate, testtime 
    #  ''',con=conn,params=[datetime_start, datetime_end],)
    # measurement_data.columns = ['SerialNo', 'TestType', 'devicecode', 'DeviceType', 'TestRig',
    #                             'TestDate', 'TestTime', 'Operator', 'ItemNo', 'ItemNoRev', 'Passed',
    #                             'FuseOperator', 'FuseRig', 'FuseDate']
    # measurement_table_tap10 = pd.merge(measurement_data,measurement_TAP10,on='devicecode', how='inner')
    # measurement_table_WDM = pd.merge(measurement_data,measurement_WDM,on='devicecode', how='inner')
    # measurement_table_3dB = pd.merge(measurement_data,measurement_3dB,left_on='devicecode', right_on='Devicecode', how='inner')
    # measurement_table_all_product = pd.concat([measurement_table_tap10, measurement_table_WDM,measurement_table_3dB], axis=0)
    # measurement_table_all_product = measurement_table_all_product.sort_values(by=['TestRig','TestDate','TestTime'], ascending=False)
    # #create datetime column
    # measurement_table_all_product["DateTime_test"] = pd.to_datetime(measurement_table_all_product['TestDate'].apply(str)+" "+measurement_table_all_product['TestTime'], infer_datetime_format=True)
    # 
    # measurement_table_all_product =measurement_table_all_product.query('(DateTime_test>=@datetime_start and DateTime_test<=@datetime_end)')
    
    #---------------------------------------------------------------RigData table-------------------------------------------
    print(f'datetime_start is {datetime_start} , datetime_end is {datetime_end}')
    rigdata = pd.read_sql_query('''SELECT *  FROM rigdata
    WHERE rigdate BETWEEN ? AND ? 
    AND (serialno NOT LIKE 'F%')
    
    ''', con=conn,params=[datetime_start, datetime_end],)
    #create datetime column-
    rigdata["DateTime_start"] = pd.to_datetime(rigdata['rigdate'].apply(str)+" "+rigdata['StartTime'], infer_datetime_format=True)
    rigdata["DateTime_end"] = pd.to_datetime(rigdata['rigdate'].apply(str)+" "+rigdata['timemade'], infer_datetime_format=True)
    # rigdata =rigdata.query('(DateTime_start>=@datetime_start and DateTime_start<=@datetime_end)')
    rigdata =rigdata.sort_values(by=["rig","DateTime_start"])
    rigdata['time_made']= (rigdata['DateTime_end']-rigdata['DateTime_start']).dt.seconds / 60.0

    output_table = rigdata.groupby(by = ["rig"]).agg(
        {
            'serialno':'count',
            'time_made' :'mean'
        }
    ).reset_index()

 #------------------------------------------------------------scrap data-----------------------------
    scrap_table = scrap.dropna().groupby(by = ["rigid"]).agg(
        {
            'scrap_description':'count'
        }
    ).reset_index()
    scrap_table.rename(columns={'rigid':'rig'},inplace=True)
    donwtime_table =downtime[["rig","rigtime","type"]]
    donwtime_table['time_delta'] =(pd.to_timedelta(donwtime_table['rigtime'].str.strip())).dt.seconds / 60

    final_yield_table = pd.merge(output_table,scrap_table,left_on='rig',right_on='rig',how='outer')
    final_yield_table["scrap_description"]=final_yield_table["scrap_description"].fillna(0)
    final_yield_table= final_yield_table[['rig','serialno','scrap_description','time_made']]
    final_yield_table = final_yield_table.assign(
        Attemp = lambda x: (x.scrap_description)+(x.serialno),
        Yield= lambda  x: round(x.serialno/(x.Attemp)*100,2)
    )
    final_yield_table['time_made'] = final_yield_table['time_made'].map('{:.2f}'.format).astype(float)
    final_yield_table.columns = ['Rig','Output','Scrap','Avg_timemade/Device','Attemp','Yield']
    final_yield_table= final_yield_table.assign(
        total_work_hour = round(total_hours*0.92),
        # crucible warm and and fusion check
        routine_rig_maintenance= 55,
        Expected_Attemp = lambda x:((x.total_work_hour-x.routine_rig_maintenance/60)/(x["Avg_timemade/Device"]/60)).apply(np.floor)

    )
    sum_stat_row = ["summary", final_yield_table['Output'].sum(), final_yield_table['Scrap'].sum(),round(final_yield_table['Avg_timemade/Device'].mean(),2),final_yield_table['Attemp'].sum(),round(final_yield_table['Yield'].mean(),2),final_yield_table['total_work_hour'].mean(),final_yield_table['routine_rig_maintenance'].sum(),final_yield_table['Expected_Attemp'].sum()]
    final_yield_table_summary = final_yield_table.copy()  #  use method copy other final_yield_table is going to affected wwhen make chagne to final_yield_table_summary
    final_yield_table_summary.loc[len(final_yield_table_summary)] = sum_stat_row


    return rigdata,measurement_table_all_product,downtime,scrap,final_yield_table,final_yield_table_summary,total_hours
    
rigdata,measurement_table_all_product,downtime,scrap,final_yield_table,final_yield_table_summary,total_hours=sql_connection(lastweek,datetime_now)



ODBC Driver 17 for SQL Server
Total hours between the two dates (excluding Saturdays from 19:00 to Sundays 19:00): 604


C:\Users\suanlamai\AppData\Local\Temp\ipykernel_27096\4141937589.py:94: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.

C:\Users\suanlamai\AppData\Local\Temp\ipykernel_27096\4141937589.py:97: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.

C:\Users\suanlamai\AppData\Local\Temp\ipykernel_27096\4141937589.py:99: UserWarning:

The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.

C:\Users\suanlamai\AppData\Local\Temp\ipykernel_27096\4141937589.py:99: UserWarning:

Could not infer format, so 

datetime_start is 2024-04-18 07:00:00 , datetime_end is 2024-04-18 18:59:00


C:\Users\suanlamai\AppData\Local\Temp\ipykernel_27096\4141937589.py:157: UserWarning:

The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.

C:\Users\suanlamai\AppData\Local\Temp\ipykernel_27096\4141937589.py:158: UserWarning:

The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.

[2024-04-18 07:30:22,470] ERROR in app: Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "C:\Users\suanlamai\Desktop\GandH\0.FO\python project\venv\Lib\site-packages\flask\app.py", line 1463, in wsgi_app
    response = self.full_dispatch_request()
               ^^^^^^^^^^^^^^^^^^^^^^^^

ODBC Driver 17 for SQL Server
Total hours between the two dates (excluding Saturdays from 19:00 to Sundays 19:00): 12


...

C:\Users\suanlamai\AppData\Local\Temp\ipykernel_27096\4141937589.py:94: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.

C:\Users\suanlamai\AppData\Local\Temp\ipykernel_27096\4141937589.py:97: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.

C:\Users\suanlamai\AppData\Local\Temp\ipykernel_27096\4141937589.py:99: UserWarning:

The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.

C:\Users\suanlamai\AppData\Local\Temp\ipykernel_27096\4141937589.py:100: UserWarning:

The argument 'infer_dateti

ODBC Driver 17 for SQL Server
Total hours between the two dates (excluding Saturdays from 19:00 to Sundays 19:00): 12
ODBC Driver 17 for SQL Server
Total hours between the two dates (excluding Saturdays from 19:00 to Sundays 19:00): 12
ODBC Driver 17 for SQL Server
Total hours between the two dates (excluding Saturdays from 19:00 to Sundays 19:00): 12
ODBC Driver 17 for SQL Server
Total hours between the two dates (excluding Saturdays from 19:00 to Sundays 19:00): 12
ODBC Driver 17 for SQL Server
Total hours between the two dates (excluding Saturdays from 19:00 to Sundays 19:00): 12
ODBC Driver 17 for SQL Server
Total hours between the two dates (excluding Saturdays from 19:00 to Sundays 19:00): 12
ODBC Driver 17 for SQL Server
Total hours between the two dates (excluding Saturdays from 19:00 to Sundays 19:00): 12
ODBC Driver 17 for SQL Server
Total hours between the two dates (excluding Saturdays from 19:00 to Sundays 19:00): 12
ODBC Driver 17 for SQL Server
Total hours between the tw

C:\Users\suanlamai\AppData\Local\Temp\ipykernel_27096\4141937589.py:94: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.

C:\Users\suanlamai\AppData\Local\Temp\ipykernel_27096\4141937589.py:97: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.

C:\Users\suanlamai\AppData\Local\Temp\ipykernel_27096\4141937589.py:99: UserWarning:

The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.

C:\Users\suanlamai\AppData\Local\Temp\ipykernel_27096\4141937589.py:100: UserWarning:

The argument 'infer_dateti

datetime_start is 2024-03-19 07:29:41 , datetime_end is 2024-04-18 07:29:41


C:\Users\suanlamai\AppData\Local\Temp\ipykernel_27096\4141937589.py:157: UserWarning:

The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.

C:\Users\suanlamai\AppData\Local\Temp\ipykernel_27096\4141937589.py:157: UserWarning:

Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.

C:\Users\suanlamai\AppData\Local\Temp\ipykernel_27096\4141937589.py:158: UserWarning:

The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.

C:\Users\suanlamai\AppData\Local\Temp\ipykernel_27096\4141937589.

In [18]:
pil_img = Image.open("assets/GandH logo.png")

# Using base64 encoding and decoding
def b64_image(image_filename):
    with open(image_filename, 'rb') as f:
        image = f.read()
    return 'data:image/png;base64,' + base64.b64encode(image).decode('utf-8')


app = dash.Dash(
    __name__,
    meta_tags=[{"name": "viewport", "content": "width=device-width, initial-scale=1"}],
)
app.title = "FO Manufacturing SPC Dashboard"
server = app.server
app.config["suppress_callback_exceptions"] = True

# APP_PATH = str(pathlib.Path().resolve().parent)
# df = pd.read_csv(os.path.join(APP_PATH, os.path.join("dash-manufacture-spc-dashboard","data", "spc_data.csv")))

def build_banner():
    print("build_banner function is called")
    return html.Div(
        id="banner",
        className="banner",
        children=[
            html.Div(
                id="banner-text",
                children=[
                    html.H5("FO Manufacturing SPC Dashboard"),
                    html.H6("Process Control and Scraps Reporting"),
                ],
            ),
            html.Div(
                id="banner-logo",
                children=[
                    html.Button(
                        id="learn-more-button", children="User Guide", n_clicks=0
                    ),
                    html.A(
                        html.Img(id="logo", src=dash.get_asset_url("GandH logo.png")),
                        href="https://gandh.com/",
                    ),
                ],
            ),
        ],
    )

def build_tabs():
    print("build_tabs function is called")
    return html.Div(
        id="tabs",
        className="tabs",
        children=[
            dcc.Tabs(
                id="app-tabs",
                value="tab2",
                className="custom-tabs",
                children=[
                    dcc.Tab(
                        id="Specs-tab",
                        label="Data Analysis",
                        value="tab1",
                        className="custom-tab",
                        selected_className="custom-tab--selected",
                        children=[build_Date_picker_and_stat_panel(total_hours),build_yield_table(),build_scrap_pareto(),build_downtime_pareto(),build_chart_panel()]
                    ),
                    dcc.Tab(
                        id="Control-chart-tab",
                        label="Production Monitor Dashboard",
                        value="tab2",
                        className="custom-tab",
                        selected_className="custom-tab--selected",
                        children=[create_timeline()]
                    #    create_timeline
                        
                    ),
                ],
            )
        ],
    )
# def build_tab_1():
#     return [
#         html.Div(
#             id="data_analysis",
#             children=[build_Date_picker_and_stat_panel(total_hours),build_yield_table(),build_scrap_pareto(),build_downtime_pareto(),build_chart_panel()],
#         ),
#        
#     ]
# 
# def build_tab_2():
#     return [
#         html.Div(
#             id="Production_line_monitor",
#             children=[create_timeline()]
#         ),
#         
# 
#     ]

def build_Date_picker_and_stat_panel(total_hours):
    
    if final_yield_table['Attemp'].sum() < round(total_hours*1.33*6*(1-1/12)):
        color = 'red'
    
    else:
        color = 'green'

    return  html.Div(
        
        children=[
            html.Div(
                id="start_date_select",
                children=[
                    html.Label(id="select_start_and_end_date", children="Select StartDate and EndDate"),
                    html.Br(),
                    dcc.DatePickerRange(
                        id="startDate_picker",
                        start_date_placeholder_text="Start Period",
                        end_date_placeholder_text="End Period",
                        calendar_orientation='vertical',
                    )
                ],style={'width': '30%', 'display': 'inline-block','vertical-align': 'top', 'margin-left': '0vw', 'margin-top': '1vw','margin-bottom': '3vw'}
                
            ),
            html.Div(

                id="total machine capacity",  # Added an ID to the div
                children=[
                    html.P(f"Rigs capacity for {total_hours*6} hrs"),
                    daq.LEDDisplay(
                        # id="operator-led",
                        value= round(total_hours*1.33*6*(1-1/12)),
                        color="#92e0d3",
                        backgroundColor="#1e2130",
                        size=30,
                    ),
                ]
                ,style={'width': '20%', 'display': 'inline-block','vertical-align': 'top','horizontal-align': 'center', 'margin-left': '0vw', 'margin-top': '1vw','margin-bottom': '3vw'}
            ),
            html.Div(

                id="total Attemps",  # Added an ID to the div
                children=[
                    html.P(f"Total Attemps "),
                    daq.LEDDisplay(
                        # id="operator-led",
                        value= round(final_yield_table['Attemp'].sum()),
                        color= color,
                        backgroundColor="#1e2130",
                        size=30,
                    ),
                ]
                ,style={'width': '15%', 'display': 'inline-block','vertical-align': 'top','horizontal-align': 'center', 'margin-left': '0vw', 'margin-top': '1vw','margin-bottom': '3vw'}
            ),
            html.Div(

                id="total Device Made",  # Added an ID to the div
                children=[
                    html.P(f"Total Output "),
                    daq.LEDDisplay(
                        # id="operator-led",
                        value= round(final_yield_table['Output'].sum()),
                        color= color,
                        backgroundColor="#1e2130",
                        size=30,
                    ),
                ]
                ,style={'width': '15%', 'display': 'inline-block','vertical-align': 'top','horizontal-align': 'center', 'margin-left': '0vw', 'margin-top': '1vw','margin-bottom': '3vw'}
            ),
            html.Div(

                id="total downtime",  # Added an ID to the div
                children=[
                    html.P(f"Total Downtime(hrs)"),
                    daq.LEDDisplay(
                        # id="operator-led",
                        value= round(downtime['dlength'].sum()/3600),
                        color= color,
                        backgroundColor="#1e2130",
                        size=30,
                    ),
                ]
                ,style={'width': '15%', 'display': 'inline-block','vertical-align': 'top','horizontal-align': 'center',  'margin-left': '0vw', 'margin-top': '1vw','margin-bottom': '3vw'}
            )
            

        ]
    )



def build_yield_table():
    return html.Div([
        dash_table.DataTable(
            id='datatable-interactivity',
            columns=[
                {"name": col, "id": col, "deletable": True, "selectable": True, "hideable": False}
                if col == "Rig" 
                else {"name": col, "id": col,}
                for col in final_yield_table_summary.columns
            ],
            style_cell={'textAlign': 'center','minWidth': 95, },
            style_data={
                'color': 'black',
                'backgroundColor': 'rgb(164,163,252)',
                'height': 'auto',
                'width': 'auto'
            },
            style_header={
                'backgroundColor': 'rgb(210, 210, 210)',
                'color': 'black',
                'fontWeight': 'bold',
                'border': '1px solid black'
            },
            style_data_conditional=[
                {
                    'if': {'row_index': 'odd'},
                    'backgroundColor': 'rgb(116,115,251)',
                }
            ],
            data=final_yield_table_summary.to_dict('records'),  # the contents of the table
            editable=False,
            cell_selectable=True,# allow editing of data inside all cells
            filter_action="native",     # allow filtering of data by user ('native') or not ('none')
            sort_action="native",       # enables data to be sorted per-column by user or not ('none')
            sort_mode="multi",         # sort across 'multi' or 'single' columns
            column_selectable="multi",  # allow users to select 'multi' or 'single' columns
            row_selectable="multi",     # allow users to select 'multi' or 'single' rows
            row_deletable=False,         # choose if user can delete a row (True) or not (False)
            selected_columns=[],        # ids of columns that user selects
            selected_rows=[],           # indices of rows that user selects
            page_action="native",       # all data is passed to the table up-front or not ('none')
            page_current=0,             # page number that user is on
            page_size=7,                # number of rows visible per page
           
        )
    ],style={'display': 'inline-block','vertical-align': 'top', 'margin-left': '0vw', 'margin-top': '1vw'}
    )


def build_scrap_pareto():
    scrap_pareto_data = scrap.dropna().query('itemno == "30163428" or worksid == "30163429" or worksid == "30183303" or worksid == "30185265" or worksid == "30184757" ')
    scrap_pareto_data=scrap_pareto_data.groupby(by=['rigid','scrap_description',]).agg(
        {
            'operator':'count'
        }
    ).rename(columns={'operator':'count'}).reset_index()

    scrap_pareto_data.sort_values(by=['rigid','count'],inplace=True)

    rig_list = [rig for rig in rigdata['rig'].unique()]
    
    y_axis_max= scrap_pareto_data["count"].max() + 1
    width = 100/len(rig_list)

    graphs=[]
    for rig in rig_list:

        
        scrap_table = scrap_pareto_data.query('rigid ==@rig')
        scrap_table["cumpercentage"] = scrap_table["count"].cumsum()/scrap_table["count"].sum()*100
        scrap_table["cumpercentage"] = round(scrap_table["cumpercentage"],2)
        scrap_table.sort_values(by='cumpercentage',ascending=False,inplace=True)
        scrap_table=scrap_table.head(5)

        fig = go.Figure([go.Bar(x=scrap_table['scrap_description'], y=scrap_table['count'], yaxis='y1', name='count'),
                         go.Scatter(x=scrap_table['scrap_description'], y=scrap_table['cumpercentage'], yaxis='y2', name='cumulative %',
                                    hovertemplate='%{y}', marker={'color': 'red'})]).update_layout(template='plotly_white', showlegend=False, hovermode='x', bargap=.3,
                                                                                                   title=dict(text= f'<b>{rig} Fail <br>Pareto<b> ', x= .5,
                                                                                                              font={'size' : 14}),
                                                                                                   yaxis={'title': 'count'},
                                                                                                   yaxis2={'rangemode': "tozero", 'overlaying': 'y',
                                                                                                           'position': 1, 'side': 'right',
                                                                                                           'title': 'cumulative %',
                                                                                                           'tickmode': 'array',
                                                                                                           'ticktext': [str(i) + '%' for i in range(0, 101, 20)]},
                                                                                                   margin=dict(l=1, r=1, t=60, b=30),
                                                                                                   yaxis_range=[0,y_axis_max],
                                                                                                   height = 350

                                                                                                   )
        fig.update_xaxes(tickangle=45)
        # fig_list.append(fig)

        graph = dcc.Graph(

            figure= fig
        )

        # width = 100/len(rig_list)     
        #
        # output_list = []

        graphs.append(graph)
    graph_rows = []
    for i in range(0, len(graphs), 3):
        row = html.Div(
            [html.Div(graph, style={'width': '33.33%', 'display': 'inline-block'}) for graph in graphs[i:i+3]],
            style={'display': 'flex','margin-top': '20px'}
        )
        graph_rows.append(row)
    return   html.Div(graph_rows)

    # Layout of the app with the graphs aligned horizontally
    # return      html.Div(
    #     children=[
    #         html.Div(
    #             children=graph,
    #             style={'width': f'{width}%', 'display': 'inline-block','margin': '5px'}
    #         ) for graph in graphs
    #     ],
    #     style={'display': 'flex', 'flex-wrap': 'wrap',}
    # )


def build_downtime_pareto():
    downtime_pareto_data = downtime.copy()
    downtime_pareto_data=downtime_pareto_data.groupby(by=['rig','type',]).agg(
        {
            'dlength':'sum'
        }
    ).rename(columns={'dlength':'sum'}).reset_index()
    downtime_pareto_data =downtime_pareto_data.assign(
        sum = lambda x:round((x["sum"]/3600),2)
    )

    rig_list = [rig for rig in downtime_pareto_data['rig'].unique()]
    width = 100/len(rig_list)
   
    y_axis_max= downtime_pareto_data["sum"].max() + 1

    graphs = []
    for rig in rig_list:
        
        downtime_pareto_data.sort_values(by=['rig','sum'],inplace=True)
        downtime_table = downtime_pareto_data.query('rig ==@rig ')
        downtime_table["cumpercentage"] = round(downtime_table["sum"].cumsum()/downtime_table["sum"].sum()*100,2)
        
        downtime_table.sort_values(by='cumpercentage',ascending=False,inplace=True)
        downtime_table=downtime_table.head(4)

        fig = go.Figure([go.Bar(x=downtime_table['type'], y=downtime_table['sum'], yaxis='y1', name='count',marker=dict(color='rgb(33,142,174)')),
                         go.Scatter(x=downtime_table['type'], y=downtime_table['cumpercentage'], yaxis='y2', name='cumulative %',
                                    hovertemplate='%{y}', marker={'color': 'red'})]).update_layout(template='plotly_white', showlegend=False, hovermode='x', bargap=.3,
                                                                                                   title=dict(text= f'<b>{rig} Downtime <br>Pareto<b> ', x= .5,
                                                                                                              font={'size':14}),
                                                                                                   yaxis={'title': 'Downtime(hrs)'},
                                                                                                   yaxis2={'rangemode': "tozero", 'overlaying': 'y',
                                                                                                           'position': 1, 'side': 'right',
                                                                                                           'title': 'cumulative %',
                                                                                                           'tickmode': 'array',
                                                                                                           'ticktext': [str(i) + '%' for i in range(0, 101, 20)]},
                                                                                                   yaxis_range=[0,y_axis_max],
                                                                                                   
                                                                                                   margin=dict(l=1, r=1, t=60, b=30),
                                                                                                   height = 350

                                                                                                   )
        fig.update_xaxes(tickangle=45)
        # fig_list.append(fig)
        graph = dcc.Graph(
            
            figure= fig
        )
 
        graphs.append(graph)
    graph_rows = []
    for i in range(0, len(graphs), 3):
        row = html.Div(
            [html.Div(graph, style={'width': '33.33%', 'display': 'inline-block'}) for graph in graphs[i:i+3]],
            style={'display': 'flex','margin-top': '20px'}
        )
        graph_rows.append(row)
    return   html.Div(graph_rows)

    



products = [product for product in rigdata.itemnumber.unique()]

parameter_list_selected = ['cratio1','cratio2','cratio3','cratio4','cratio5','cratio6','cratio7','cratio8','cratio9','extension']
def build_parameter_select_dropdown():
     return html.Div([
            html.Div(id="parameter-select-menu",
                     children=[
                         html.Label(id="select Paremeter", children="Select Parameter"),
                         html.Br(),
                         dcc.Dropdown(
                             id="parameter_dropdown",
                             options=list(
                                 {"label": param, "value": param} for param in parameter_list_selected
                             ),
                             value="extension",
                         ),
                         
                     ], style={'display': 'inline-block', "width": "20%",'vertical-align': 'top', 'margin-left': '3vw', 'margin-top': '3vw','margin-bottom': '3vw'}),
            html.Div( id="product-select-menu",
                      children=[
                          html.Label(id="select Product", children="Select product"),
                          html.Br(),
                          dcc.Dropdown(
                              id="product_dropdown",
                              options=list(
                                  {"label": product, "value": product} for product in products
                              ),
                              value="30163428",
                          ),
                          ],style={'display': 'inline-block',"width": "15%", 'vertical-align': 'top', 'margin-left': '3vw', 'margin-top': '3vw','margin-bottom': '3vw'},
  
            ),
  
     ],style={'display': 'inline-block',"width": "100%", 'vertical-align': 'top', 'margin-left': '3vw', 'margin-top': '3vw','margin-bottom': '3vw'},
        
    )


rig_list =  [rig for rig in rigdata["rig"].unique()]

def build_chart_panel():
    
    
    return html.Div(
        id="control-chart-container",
        className="twelve columns",
        children=[build_parameter_select_dropdown(),
                  # generate_section_banner("Live SPC Chart"),
                  
                  dcc.Graph(
                      id="plot_parameter",
                  ),
                  
                  ],
    )

def update_gantt():

    # Add a custom JavaScript function to set the range
    day_time_start = datetime.now().replace(hour=7, minute=0, second=0, microsecond=0)
    day_time_end = datetime.now().replace(hour=18, minute=59, second=0, microsecond=0)

    night_time_start = datetime.now().replace(hour=19, minute=0, second=0, microsecond=0)
    night_time_end = datetime.now().replace(hour=19, minute=0, second=0, microsecond=0) + timedelta(hours=12)



    if (datetime.now() > day_time_start) | (datetime.now() < day_time_end):
        datetime_start = datetime.strptime(day_time_start.strftime("%Y-%m-%d %H:%M:%S"),"%Y-%m-%d %H:%M:%S")
        datetime_end = datetime.strptime(day_time_end.strftime("%Y-%m-%d %H:%M:%S"),"%Y-%m-%d %H:%M:%S")
    elif (datetime.now() > night_time_start) | (datetime.now() < night_time_end):
        datetime_start = datetime.strptime(night_time_start.strftime("%Y-%m-%d %H:%M:%S"),"%Y-%m-%d %H:%M:%S")
        datetime_end = datetime.strptime(night_time_end.strftime("%Y-%m-%d %H:%M:%S"),"%Y-%m-%d %H:%M:%S")

    else:
        pass


    rigdata,measurement_table_all_product,downtime,scrap,final_yield_table,final_yield_table_summary,total_hours=sql_connection(datetime_start,datetime_end)

    # print(f'date_from  {date_from}  dateTime_to {dateTime_to}')
    df_good_device = rigdata[['rig','DateTime_start','DateTime_end','serialno']]
    df_good_device['status'] = 'Passed'
    df_good_device.rename(columns={'rig':'Task','DateTime_start':'Start','DateTime_end' : 'Finish' },inplace=True)


    df_downtime = downtime[['rig','DateTime_start','DateTime_end','type']]
    df_downtime.rename(columns={'rig':'Task','DateTime_start':'Start','DateTime_end' : 'Finish' ,'type' : 'status'},inplace=True)

    df_scrap = scrap[['rigid','DateTime_start','DateTime_end']].dropna()
    df_scrap['status'] = 'failed'
    df_scrap.rename(columns={'rigid':'Task','DateTime_start':'Start','DateTime_end' : 'Finish' ,'type' : 'status'},inplace=True)

    df_final = pd.concat([df_good_device,df_downtime,df_scrap]).fillna('0')

    # Add one day to date1 if date1 is before date2
    df_final['Finish'] = np.where(df_final['Finish'] < df_final['Start'], df_final['Finish'] + timedelta(days=1), df_final['Finish'])
    df_final['time_made']= round((df_final['Finish']-df_final['Start']).dt.seconds / 60.0,1)




    # Task, Start, Finish
    def condition(df):
        if df == 'GLUE CURE TIME':
            return 'other'
        elif df == 'CRUCIBLE WARM' :
            return 'other'
        elif df == 'SCRAP FUSION CHECK' :
            return 'other'
        elif df == 'CRUCON' :
            return 'other'
        elif df == 'RIG CLEAN' :
            return 'other'
        elif df == 'FIBRE SPLICING' :
            return 'other'
        elif df == 'SCRAP INDEPENDENT CK' :
            return 'other'
        elif df == 'TAPER CHECK' :
            return 'other'
        elif df == 'FUSION CHECK' :
            return 'other'
        else:
            return df


    # Apply the function to create a new column
    df_final['group_status'] = df_final['status'].apply(condition)
    # df_final_copy = df_final.copy()
    # df_final_copy = np.where((df_final_copy['Start']>= datetime_start) | (df_final_copy['Start']>= datetime_end))

    # Create an empty list to store dictionaries
    list_of_dicts = []

    # Iterate over DataFrame rows and add columns as values to dictionaries
    for index, row in df_final.iterrows():

        # dict_row = {'A': row['A'], 'B': row['B']}
        dict_row = dict(Task=row['Task'], Start=row['Start'], Finish=row['Finish'], Resource=row['group_status'],Description=[row['serialno'],row['time_made']],showgrid_y = True)
        list_of_dicts.append(dict_row)

    # colors = [f'rgb{next(color_cycle)}' for _ in range(len(list_resource))]
    colors ={
        'Passed' :'rgb(0,128,0)',
        'OPERATOR BREAK' : 'rgb(128,128,128)',
        # 'GLUE CURE TIME' : 'rgb(0,255,255)',
        # 'FUSION CHECK': 'rgb(255,165,0)',
        'RIG MAINT' : 'rgb(210,105,30)',
        # 'CRUCIBLE WARM' :'rgb(0,0,255)',
        # 'SCRAP FUSION CHECK' : 'rgb(173,255,47)',
        # 'CRUCON' : 'rgb(128,0,128)',
        # 'RIG CLEAN' : 'rgb(255,0,255)',
        # 'FIBRE SPLICING' :'rgb(0,128,128)',
        # 'SCRAP INDEPENDENT CK' :'rgb(255,105,180)',
        'failed' : 'rgb(255,0,0)',
        # 'TAPER CHECK' : 'rgb(255,255,0)'
        'other' : 'rgb(255,235,205)'

    }






    fig = ff.create_gantt(list_of_dicts, colors=colors, index_col='Resource', show_colorbar=True,group_tasks=True,title = dict(x=0.5,text='<b>Rig Montor<b>'),
                          showgrid_y=False,showgrid_x=True

                          )
    fig.update_xaxes(dtick = 3600000)
    fig.layout.xaxis.rangeselector = None

    # Set the layout with a range selector
    fig.update_layout(
        xaxis=dict(
            rangeselector=dict(
                buttons=list([
                    dict(count=1,
                         label="1d",
                         step="day",
                         stepmode="backward"),

                    # dict(step="all"),

                ])
            ),
            rangeslider_visible=True,
            type="date"
        )
    )



    fig.update_layout(
        updatemenus=[
            dict(
                type="buttons",
                buttons=[
                    dict(label="<b>D-Shift<b>",
                         method="relayout",
                         args=[{"xaxis.range": [datetime_start,
                                                datetime_end]}])
                ],
                x=0.07,
                y=1.25,
                # xanchor='right',
                # yanchor='top'
            ),
            dict(
                type="buttons",
                buttons=[
                    dict(label="<b>N-Shift<b>",
                         method="relayout",
                         args=[{"xaxis.range": [datetime_start,
                                                datetime_end]}])
                ],
                x=0.15,
                y=1.25,
                # xanchor='right',
                # yanchor='top'
            )
        ]
    )

    # fig.update_xaxes(range= x_range)
    return fig
def create_timeline():

    
      
    return html.Div(
        id="timeline",
        className="twelve columns",
        children=[
            html.Div(id='last-update-message', style={'margin-top': 20})
        ,
            dcc.Graph(id='timeline-output', ),
                   
                  ],
    )
   





app.layout = html.Div(
    id="big-app-container",
    children=[
        build_banner(),
        #dcc.Interval is a component that will fire a callback periodically. Use dcc.Interval to update your app in realtime without needing to refresh the page or click on any buttons.
        dcc.Interval(
            id="interval-component",
            interval=5 * 1000,  # in milliseconds
            n_intervals=0,  # 
            # disabled=True,
        ),
        html.Div(
            id="app-container",
            children=[
                build_tabs(),
                # Main app
                html.Div(id="app-content"),
            ],
        ),
    ],
)

# @app.callback(
#     Output('plot_parameter', 'figure'),
#     [Input('date-picker-range', 'start_date'),
#      Input('date-picker-range', 'end_date')])
# def update_data_from_database(start_date,end_date):
#     
#     rigdata,measurement_table_all_product,downtime,scrap,final_yield_table,final_yield_table_summary,total_hours=sql_connection(start_date,end_date)
#     build_tabs()
    
@app.callback(
    Output('plot_parameter', 'figure'),
    [Input('parameter_dropdown', 'value'),
    Input('product_dropdown', 'value')])
def update_plot(selected_parameter, selected_product):


    # & itemnumber== "30163428"
    df = rigdata.sort_values(by='DateTime_start').query( 'itemnumber == @selected_product'  )

    return  go.Figure(
            {
                "data": [
                    {

                        "x" : df.query('rig== @rig')['DateTime_start'].sort_values() ,
                        "y" : df.query( 'rig== @rig' )[f'{selected_parameter}'],

                        "mode": "lines+markers",
                        "name": rig
                    } for rig in rig_list
                ],
                "layout": { 'title' : {'text':f"{selected_parameter} of product {selected_product}",'x':0.5 } ,
                            "paper_bgcolor": 'white',
                            "plot_bgcolor": 'rgb(227,227,254)',
                            "xaxis": dict(
                                showline=False, showgrid=False, zeroline=False,title="Date"
                            ),
                            "yaxis": dict(
                                showgrid=False, showline=False, zeroline=False,title=f"{selected_parameter}"
                            ),
                            "autosize": True,
                            },
            }
        )




@app.callback(
    
    Output('last-update-message', 'children'),
    Output('timeline-output', 'figure'),
    
    Input('interval-component', 'n_intervals')
)
def update_Gantt_data(n):
    



    # date_from = '2024-03-21'
    # Time_from ='07:00:00'
    # datetime_start = pd.to_datetime(date_from+" "+Time_from, infer_datetime_format=True)
    # 
    # date_to = '2024-03-22'
    # Time_to ='07:00:00'
    # datetime_end = pd.to_datetime(date_to+" "+Time_to, infer_datetime_format=True)
    # 
    # rigdata,measurement_table_all_product,downtime,scrap,final_yield_table,final_yield_table_summary,total_hours=sql_connection(datetime_start,datetime_end)
    fig=update_gantt()
   
    # Create the message box content
    message = f"Last updated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}"

    # Create the figure
    # print(fig)
    return message,fig

if __name__ == "__main__":
    app.run_server()
    # app.run_server(debug=True, port=8050)
    

    

build_banner function is called
build_tabs function is called


In [5]:
downtime

,rig,operator,rigdate,rigtime,type,dlength,seqcomplete,seqendphase,fcheckar,fcheckpass,...,pyrometer,crucibledevices,curetime,PK_GUID,ConCheckDone,ConCheckPassed,StartTime,fcheckshape,DateTime_start,DateTime_end
28415,FFR02,sartphet,2024-03-25,23:03:36,FUSION CHECK,5163,True,16,1.25,True,...,3.163219,31,0.000,87EE26B7-BAEA-EE11-BF60-00155D6EFD00,True,True,21:37:33,None,2024-03-25 21:37:33,2024-03-25 23:03:36
28416,FFR02,sartphet,2024-03-26,00:17:44,OPERATOR BREAK,4433,False,0,0.00,False,...,0.000000,31,0.000,7C908612-C5EA-EE11-BF60-00155D6EFD00,False,False,23:03:51,None,2024-03-26 23:03:51,2024-03-26 00:17:44
28417,FFR02,sartphet,2024-03-26,00:30:28,GLUE CURE TIME,664,False,0,0.00,False,...,0.000000,31,1.453,282AFDD9-C6EA-EE11-BF60-00155D6EFD00,False,False,00:19:24,None,2024-03-26 00:19:24,2024-03-26 00:30:28
28418,FFR02,sartphet,2024-03-26,01:31:30,FUSION CHECK,3652,True,16,1.35,True,...,3.143280,32,0.000,0BD5FA5B-CFEA-EE11-BF60-00155D6EFD00,True,True,00:30:38,None,2024-03-26 00:30:38,2024-03-26 01:31:30
28419,FFR02,sartphet,2024-03-26,02:57:00,FUSION CHECK,5124,True,16,1.40,True,...,3.453841,33,0.000,E7B0A752-DBEA-EE11-BF60-00155D6EFD00,True,True,01:31:37,None,2024-03-26 01:31:37,2024-03-26 02:57:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28511,FFR02,pmangmee,2024-04-03,00:27:30,GLUE CURE TIME,582,False,0,0.00,False,...,0.000000,9,1.313,4C166AC7-0FF1-EE11-BF60-00155D6EFD00,False,False,00:17:49,None,2024-04-03 00:17:49,2024-04-03 00:27:30
28512,FFR02,pmangmee,2024-04-03,01:37:46,FUSION CHECK,4188,True,16,1.35,True,...,3.703600,10,0.000,A7900798-19F1-EE11-BF60-00155D6EFD00,True,True,00:27:57,None,2024-04-03 00:27:57,2024-04-03 01:37:46
28513,FFR02,pmangmee,2024-04-03,03:37:08,CRUCON,1894,True,21,0.00,False,...,3.434894,0,0.000,59845045-2AF1-EE11-BF60-00155D6EFD00,False,False,03:05:34,None,2024-04-03 03:05:34,2024-04-03 03:37:08
28514,FFR02,pmangmee,2024-04-03,04:13:40,FUSION CHECK,2152,True,16,1.30,True,...,3.675142,1,0.000,D188E75F-2FF1-EE11-BF60-00155D6EFD00,True,True,03:37:49,None,2024-04-03 03:37:49,2024-04-03 04:13:40
